<a href="https://colab.research.google.com/github/ryanjameskim/public/blob/master/210427_ETF2Vec_Batch_Implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
import io
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.sequence import skipgrams
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.layers import Reshape, Embedding, Dense, Dot, Flatten
from tensorflow.keras import Model



In [3]:
from google.colab import drive

drive.mount('/gdrive')

Mounted at /gdrive


In [4]:
df = pd.read_csv('/gdrive/MyDrive/ML/Ishares Data/210331/Holdings_small/Aggregate.csv')

In [5]:
df = df.sort_values(['ETF', 'Weight (%)'], ascending=[True, False])

#Find tickers and frequencies
val_cnts = df['Ticker'].value_counts() 
etf_names = df['ETF'].value_counts().index.to_numpy()
np.random.seed(42)
np.random.shuffle(etf_names)

#Create vocabulary and integerization of stock tickers
vocab_size = len(val_cnts)  #3866
idx_to_tick = {i + 1: ticker for i, ticker in enumerate(val_cnts.keys())}  #natural counting
idx_to_tick[0] = '<PAD>'
tick_to_idx = {ticker : index for index, ticker in idx_to_tick.items()}   #inverse

#constants
window_size = 3    # three nearest in portfolio % weight to the stock in question
vector_dim = 128    # dimensionality of the embedding vector
epochs = 10 ** 6    # one million epochs
num_ns = 10  # number of negative samples
SEED = 42 # for replication
AUTOTUNE = tf.data.AUTOTUNE # for caching


In [6]:
#***SINGLE EXAMPLE***#
#example
example_etf = [tick_to_idx[tick] for tick in df[df['ETF'] == etf_names[90]]['Ticker']]
window_size = 2
positive_skip_grams, _ = tf.keras.preprocessing.sequence.skipgrams(
      example_etf,
      vocabulary_size=vocab_size + 1,
      window_size=window_size,
      negative_samples=0)
print(len(example_etf))  # 125
print(len(positive_skip_grams))   #494 = 125 * 4 - 6

#look at example
for target, context in positive_skip_grams[:5]:
    print(f'{target}, {context}: {idx_to_tick[target]}, {idx_to_tick[context]}')

#negative sampling example
target_word, context_word = positive_skip_grams[0]

context_class = tf.reshape(tf.constant(context_word, dtype="int64"), (1, 1))

negative_sampling_candidates, _, _ = tf.random.log_uniform_candidate_sampler(
    true_classes=context_class,  # class that should be sampled as 'positive'
    num_true=1,  # each positive skip-gram has 1 positive context class
    num_sampled=num_ns,  # number of negative context words to sample
    unique=True,  # all the negative samples should be unique
    range_max=vocab_size,  # pick index of the samples from [0, vocab_size]
    seed=42,  # seed for reproducibility
    name="negative_sampling"  # name of this operation
)
#!!!Because this returns [0, vocab_size), must add +1 to get proper idx index

print(negative_sampling_candidates) #Note the zero: tf.Tensor([ 77   0 143 265 103], shape=(5,), dtype=int64)
print([idx_to_tick[idx.numpy() + 1] for idx in negative_sampling_candidates])

#manually create one training example
#expand dimensions
negative_sampling_candidates = tf.expand_dims(negative_sampling_candidates, 1)

#concatenate with positive example
context = tf.concat([context_class, negative_sampling_candidates], 0)

# Label first context word as 1 (positive) followed by num_ns 0s (negative).
label = tf.constant([1] + [0]*num_ns, dtype="int64")

# Reshape target to shape (1,) and context and label to (num_ns+1,).
target = tf.squeeze(target_word)
context = tf.squeeze(context)
label = tf.squeeze(label)


print(f"target_index    : {target}")
print(f"target_word     : {idx_to_tick[target_word]}")
print(f"context_indices : {context}")
print(f"context_words   : {[idx_to_tick[c.numpy()] for c in context]}")
print(f"label           : {label}")



125
494
578, 36: ZM, ZTS
164, 36: GRMN, ZTS
33, 225: BIIB, NVR
15, 58: EA, CDNS
578, 51: ZM, TSCO
tf.Tensor([164  18 788  55 297 504 131   0 176 826], shape=(10,), dtype=int64)
['PHM', 'DVA', 'NVCR', 'HPQ', 'XOM', 'Z', 'SCHW', 'PSA', 'HST', 'SWX']
target_index    : 578
target_word     : ZM
context_indices : [ 36 164  18 788  55 297 504 131   0 176 826]
context_words   : ['ZTS', 'GRMN', 'INTC', 'CC', 'AVGO', 'MPC', 'RGLD', 'STE', '<PAD>', 'NEM', 'USM']
label           : [1 0 0 0 0 0 0 0 0 0 0]


In [7]:
#function to create skipgrams per ETF
def generate_training_data(etf_names, window_size, num_ns, vocab_size, seed):
    # Elements of each training example are appended to these lists.
    targets, contexts, labels = [], [], []
    # Build the sampling table for vocab_size tokens.
    sampling_table = tf.keras.preprocessing.sequence.make_sampling_table(
                        vocab_size + 1) #+1 due to index zero being skipped
    #for each ETF, generate positive skip-gram
    for etf in etf_names:
        tokens = np.array([tick_to_idx[tick] for tick in df.loc[df['ETF'] == etf,
                                                            'Ticker'].values]) 
        positive_skip_grams, _ = skipgrams(
                            tokens,
                            vocab_size + 1,
                            window_size = window_size,
                            negative_samples = 0)
        #Iterate over each positive skip-gram for negative samples
        for target_word, context_word in positive_skip_grams:
            context_class = tf.expand_dims(
                                tf.constant([context_word], dtype="int64"), 1)
            negative_sampling_candidates, _, _ = tf.random.log_uniform_candidate_sampler(
                                                true_classes=context_class,
                                                num_true=1,
                                                num_sampled=num_ns,
                                                unique=True,
                                                range_max=vocab_size,
                                                seed=SEED,
                                                name="negative_sampling")
            negative_sampling_candidates += 1
            # Build context and label vectors (for one target word)
            negative_sampling_candidates = tf.expand_dims(
                                            negative_sampling_candidates, 1)
            context = tf.concat([context_class, negative_sampling_candidates], 0)
            label = tf.constant([1] + [0]*num_ns, dtype="int64")
            targets.append(target_word)
            contexts.append(context)
            labels.append(label)
    return targets, contexts, labels

targets, contexts, labels = generate_training_data(
    etf_names=etf_names,
    window_size=window_size,
    num_ns=num_ns,
    vocab_size=vocab_size,
    seed=SEED)
print(len(targets), len(contexts), len(labels))

#make dataset using tf.data.Dataset API
BATCH_SIZE = 1024
BUFFER_SIZE = 10000
dataset = tf.data.Dataset.from_tensor_slices(((targets, contexts), labels))
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
#dataset = dataset.cache().prefetch(buffer_size=AUTOTUNE)
print(dataset)



248586 248586 248586
<BatchDataset shapes: (((1024,), (1024, 11, 1)), (1024, 11)), types: ((tf.int32, tf.int64), tf.int64)>


In [8]:

#***#Keras Subclassing API Model***#
class ETF2Vec(Model):
    def __init__(self, vocab_size, embedding_dim):
        super(ETF2Vec, self).__init__()
        self.target_embedding = Embedding(vocab_size + 1,
                                          vector_dim,
                                          input_length=1,
                                          name="E2V_embedding")
        self.context_embedding = Embedding(vocab_size + 1,
                                           vector_dim,
                                           input_length=num_ns+1)
        self.dots = Dot(axes=(3, 2))
        self.flatten = Flatten()
    def call(self, pair):
        target, context = pair
        word_emb = self.target_embedding(target)
        context_emb = self.context_embedding(context)
        dots = self.dots([context_emb, word_emb])
        return self.flatten(dots)


#**Compile Model**#
etf2vec = ETF2Vec(vocab_size, vector_dim)
etf2vec.compile(optimizer='adam',
                 loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                 metrics=['accuracy'])


#Callback for Tensorboard
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="logs")

etf2vec.fit(dataset, epochs=100, callbacks=[tensorboard_callback])


Epoch 1/100
242/242 [==============================] - 5s 13ms/step - loss: 2.3385 - accuracy: 0.2224
Epoch 2/100
242/242 [==============================] - 3s 12ms/step - loss: 1.6933 - accuracy: 0.3941
Epoch 3/100
242/242 [==============================] - 3s 12ms/step - loss: 1.2511 - accuracy: 0.6129
Epoch 4/100
242/242 [==============================] - 3s 12ms/step - loss: 0.9995 - accuracy: 0.6961
Epoch 5/100
242/242 [==============================] - 3s 12ms/step - loss: 0.8634 - accuracy: 0.7402
Epoch 6/100
242/242 [==============================] - 3s 12ms/step - loss: 0.7725 - accuracy: 0.7650
Epoch 7/100
242/242 [==============================] - 3s 12ms/step - loss: 0.7028 - accuracy: 0.7835
Epoch 8/100
242/242 [==============================] - 3s 12ms/step - loss: 0.6472 - accuracy: 0.7997
Epoch 9/100
242/242 [==============================] - 3s 12ms/step - loss: 0.5986 - accuracy: 0.8155
Epoch 10/100
242/242 [==============================] - 3s 11ms/step - loss: 0.555

In [9]:

#For saving vectors and metadata
weights = etf2vec.get_layer('E2V_embedding').get_weights()[0]
vocab = ['<PAD>'] + val_cnts.keys().to_list()

out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(vocab):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()

#For use in Google Colabs
try:
  from google.colab import files
  files.download('vectors.tsv')
  files.download('metadata.tsv')
except Exception:
  pass


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [20]:
#validation constants

custom_examples = ['CSCO', 'NKE', 'INTC', 'GS', 'T', 'TSLA', 'AAPL', 'PAYX']
valid_examples = [tick_to_idx[tick] for tick in custom_examples]


In [21]:
#simple cosine similarity with resultant weights

def get_sim(valid_ex_id):
  sim = np.zeros((vocab_size,))  #initalize sim and inputs
  for i in range(vocab_size):     #for each ticker, test the validation ticker against each other stock 
      sim[i] = np.sum(weights[valid_ex_id] * weights[i])
  return sim

for i, valid_ex_id in enumerate(valid_examples):       #goes through each of the validation examples
  valid_name = idx_to_tick[valid_ex_id]       #find name of validation ticker 
  top_k = 8                                # number of nearest neighbors
  sim = get_sim(valid_ex_id)        #get similarity against all other tickers
  nearest = (-sim).argsort()[1:top_k + 1]  #gets negative of the cosine scores, lists the indexes of the most negative to positve scores
                                            #then indexing the argsort lists the smallest numbers, skips index 0 because it will be -1 (the cosine similarity to itself) 
  log_str = f'Nearest to {valid_name}:'
  for kth_near in nearest:
      close_tick = idx_to_tick[kth_near]
      log_str = f'{log_str} {close_tick},'  #keeps appending to the log string
  print(log_str)




Nearest to CSCO: XOM, GOOGL, KO, AMED, MO, CMCSA, CRM, CVX,
Nearest to NKE: HON, AAPL, WPC, ACN, ABBV, AMD, QRTEA, HLF,
Nearest to INTC: MA, NVDA, NFLX, AAPL, JNJ, MSFT, CRM, V,
Nearest to GS: DE, ETP, CFX, LMT, OUT, RWT, BLK, WPC,
Nearest to T: XOM, FTV, TEAM, JPM, RHP, FLO, NFG, AVGO,
Nearest to TSLA: FB, AMZN, GOOGL, AAPL, MSFT, WHR, UNH, INDA,
Nearest to AAPL: ACI, GOOGL, MSFT, AMZN, LSCC, FB, FAF, GOOG,
Nearest to PAYX: AZO, MTCH, AXTA, WCN, LSI, GE, DAL, TDG,
